In [1]:

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup as bs
import pandas as pd
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

In [2]:
path = "E:/Virtual environment/Recommender_Systems/ml-latest-small/"
links = pd.read_csv(path + 'links.csv')
URLs = links['imdbId']

In [3]:
search_string = "https://www.imdb.com/title/tt0"
IMDBS = URLs.values.tolist()
URL_list = []

for i in IMDBS:
    i = str(i)
    final_URL = search_string + i + "/"
    URL_list.append(final_URL)

In [4]:
URLS = URL_list[0:500]

In [5]:

service = Service(ChromeDriverManager().install())

# create the driver instance
driver = webdriver.Chrome(service=service)

# Define a function to scrape user reviews and movie title for a given URL
def scrape_reviews(url):
    # Load the URL in the Chrome webdriver
    driver.get(url)
    # Extract the movie title from the webpage
    content = driver.page_source
    soup = bs(content, 'html.parser')
    title_elem = soup.find('h1')
    
    if title_elem:
        title = title_elem.text.strip()
    else:
        title = ''
    # Wait for the user review button to be clickable and click it
    user_review_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.LINK_TEXT, 'User reviews')))
    user_review_button.click()
    # Wait for the reviews to load using an explicit wait
    try:
        load_more_button = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, 'load-more-trigger')))
        while True:
            load_more_button.click() 
            time.sleep(2) # Wait for new reviews to load
            load_more_button = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, 'load-more-trigger')))
    except:
        pass
    # Extract the reviews using BeautifulSoup
    content = driver.page_source
    soup = bs(content, 'html.parser')
    reviews = soup.find_all('div', class_='text show-more__control')
    user_reviews = [review.text.strip() for review in reviews]
    # Return the movie title and list of reviews
    return title, user_reviews

# Iterate over each URL and scrape the data
data = []
for url in URLS:
    try:
        # Scrape the movie title and reviews for the URL
        title, user_reviews = scrape_reviews(url)
        # Append the title and reviews for this URL to the data list
        data.append({'Title': title, 'User Reviews': user_reviews})
    except Exception as e:
        print(f"Error occurred while scraping data from {url}: {e}")

# Create a DataFrame from the data list and save it to a CSV file
df = pd.DataFrame(data)

# Quit the Chrome webdriver
try:
    driver.quit()
except:
    pass


In [16]:
df

,Title,User Reviews
0,Toy Story,[Toy Story – 5/5 starsChildren play with toys....
1,Jumanji,[You know the drill: young Alan Parrish gets s...


In [8]:
# x = pd.read_csv("x.csv")
# z = df.copy()
# a = pd.concat([x, z], axis=0)
# a = a.reset_index().drop(columns=['index'])
# a

In [9]:
df.to_csv('x.csv', index=False, sep=',')

In [10]:
# y = pd.read_csv("User_reviews_300to400.csv")
# y

In [11]:
# da = y.explode('User Reviews')

# da.to_csv('All_user_reviews_300to400.csv', index=False)

In [12]:
w = pd.read_csv("x.csv")
w

,Title,User Reviews
0,Toy Story,"[""Toy Story – 5/5 starsChildren play with toys..."
1,Jumanji,['You know the drill: young Alan Parrish gets ...
